# Week 7: LangGraph 에이전트 아키텍처 II - 성찰 메커니즘과 다중 에이전트 시스템

## 과제 개요
- **주제**: "AI 연구팀 시뮬레이터" 구축
- **목표**: 여러 전문 에이전트가 협업하며 자가 개선하는 시스템
- **핵심**: 성찰(Reflection) + 서브그래프(Subgraph) + 다중 에이전트(Multi-Agent)

## 6주차 vs 7주차

| 항목 | 6주차 | 7주차 |
|------|-------|-------|
| 에이전트 | 단일 에이전트 | 다중 에이전트 (3개+) |
| 개선 | 재계획만 | 성찰 + 자가 개선 |
| 구조 | 단일 그래프 | 메인 + 서브그래프 |
| 협업 | 없음 | 순차/병렬 협업 |
| 피드백 | LLM만 | 에이전트 간 피드백 |

## 비유

- **6주차**: "혼자 일하는 똑똑한 직원"
  - 계획 → 실행 → 확인 → 재계획
  
- **7주차**: "서로 피드백하며 협업하는 전문가 팀"
  - 연구원 + 작가 + 비평가가 협력
  - 각자 자기 작업을 평가하고 개선
  - 서로 결과물을 주고받으며 완성도 향상

## AI 연구팀 구조
```
        ┌─────────────┐
        │ Supervisor  │ (감독관)
        │  (메인)     │
        └──────┬──────┘
               ↓
    ┌──────────┼──────────┐
    ↓          ↓          ↓
┌────────┐ ┌────────┐ ┌────────┐
│Research│ │ Writer │ │ Critic │
│ Agent  │ │ Agent  │ │ Agent  │
└────────┘ └────────┘ └────────┘
(서브그래프)(서브그래프)(서브그래프)

각 서브그래프:
  1. receive_task (작업 받기)
  2. execute (실행)
  3. reflect (성찰)
  4. revise (개선)
  5. submit (제출)
```

## 구현 내용

1. **Section 0**: 환경 설정 (6주차 재사용)
2. **Section 1**: 전문 에이전트 정의 (3개)
3. **Section 2**: 성찰 메커니즘 구현
4. **Section 3**: 서브그래프 구성
5. **Section 4**: 다중 에이전트 협업
6. **Section 5**: 시나리오 테스트 (4개)

In [1]:
print("="*80)
print("Cell 2: 라이브러리 Import")
print("="*80)

# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# 기본 라이브러리
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

import os
from dotenv import load_dotenv
import warnings
warnings.filterwarnings('ignore')

# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# LangChain Core (6주차 재사용)
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage

# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# LangChain Tools (6주차 재사용)
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

from langchain.tools import Tool
from langchain_community.tools.tavily_search import TavilySearchResults

# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# LangGraph (6주차 + 7주차 신규)
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

from langgraph.graph import StateGraph, START, END
from langgraph.checkpoint.memory import MemorySaver
from typing import TypedDict, Annotated, Literal, List, Dict, Any, Optional
from langgraph.graph.message import add_messages

# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# 추가 라이브러리 (7주차 신규)
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

from dataclasses import dataclass
from enum import Enum
import json

print("✅ 라이브러리 Import 완료")

print("""
[7주차 새로운 Import]
  - dataclass: 에이전트 클래스 정의
  - Enum: 에이전트 타입, 작업 상태 등
  - json: 에이전트 간 데이터 전달
  
[6주차에서 재사용]
  - ChatOpenAI, PromptTemplate
  - StateGraph, MemorySaver
  - add_messages
  - TavilySearchResults
""")

Cell 2: 라이브러리 Import
✅ 라이브러리 Import 완료

[7주차 새로운 Import]
  - dataclass: 에이전트 클래스 정의
  - Enum: 에이전트 타입, 작업 상태 등
  - json: 에이전트 간 데이터 전달

[6주차에서 재사용]
  - ChatOpenAI, PromptTemplate
  - StateGraph, MemorySaver
  - add_messages
  - TavilySearchResults



In [2]:
print("\n" + "="*80)
print("Cell 3: API 키 로드")
print("="*80)

# API 키 로드
load_dotenv()

# OpenAI API
openai_api_key = os.getenv("OPENAI_API_KEY")
if not openai_api_key:
    raise ValueError("OPENAI_API_KEY not found in .env file")
print("✅ OpenAI API 키 로드 완료")

# Tavily API (웹 검색용)
tavily_api_key = os.getenv("TAVILY_API_KEY")
if tavily_api_key:
    print("✅ Tavily API 키 로드 완료")
else:
    print("⚠️ TAVILY_API_KEY not found - Research Agent 기능 제한")

print("""
[7주차 필요 API]
  - OPENAI_API_KEY (필수): 모든 에이전트의 LLM
  - TAVILY_API_KEY (권장): Research Agent의 웹 검색
""")


Cell 3: API 키 로드
✅ OpenAI API 키 로드 완료
✅ Tavily API 키 로드 완료

[7주차 필요 API]
  - OPENAI_API_KEY (필수): 모든 에이전트의 LLM
  - TAVILY_API_KEY (권장): Research Agent의 웹 검색



In [3]:
print("\n" + "="*80)
print("Cell 4: LLM 설정")
print("="*80)

# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# LLM 설정
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

LLM_MODEL = "gpt-4o-mini"
LLM_TEMPERATURE = 0.7  # 7주차는 창의성이 필요하므로 0.7

llm = ChatOpenAI(
    model=LLM_MODEL,
    temperature=LLM_TEMPERATURE,
    api_key=openai_api_key
)

print(f"✅ LLM 설정 완료: {LLM_MODEL}")

print("""
[LLM 설정 변경점]
  6주차: temperature=0 (결정적)
  7주차: temperature=0.7 (창의적)
  
  이유: Writer Agent가 다양한 표현 필요
  
[7주차에서 LLM 역할]
  1. Research Agent: 정보 분석 및 요약
  2. Writer Agent: 콘텐츠 작성
  3. Critic Agent: 평가 및 피드백
  4. 성찰: 자가 평가 및 개선점 도출
""")


Cell 4: LLM 설정
✅ LLM 설정 완료: gpt-4o-mini

[LLM 설정 변경점]
  6주차: temperature=0 (결정적)
  7주차: temperature=0.7 (창의적)

  이유: Writer Agent가 다양한 표현 필요

[7주차에서 LLM 역할]
  1. Research Agent: 정보 분석 및 요약
  2. Writer Agent: 콘텐츠 작성
  3. Critic Agent: 평가 및 피드백
  4. 성찰: 자가 평가 및 개선점 도출



In [4]:
print("\n" + "="*80)
print("Cell 5: 에이전트 타입 및 상수 정의")
print("="*80)

# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# 에이전트 타입 정의
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

class AgentType(Enum):
    """에이전트 타입"""
    RESEARCH = "research"      # 연구원
    WRITER = "writer"          # 작가
    CRITIC = "critic"          # 비평가
    SUPERVISOR = "supervisor"  # 감독관

# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# 작업 상태 정의
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

class TaskStatus(Enum):
    """작업 상태"""
    PENDING = "pending"        # 대기 중
    IN_PROGRESS = "in_progress"  # 진행 중
    REFLECTING = "reflecting"  # 성찰 중
    REVISING = "revising"      # 수정 중
    COMPLETED = "completed"    # 완료
    FAILED = "failed"          # 실패

# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# 성찰 관련 상수
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

# 성찰 최대 반복 횟수
MAX_REFLECTION_ITERATIONS = 3

# 합격 점수 기준 (1-10점)
PASS_THRESHOLD = 7

# 평가 항목
EVALUATION_CRITERIA = {
    "accuracy": "정확성",      # 정보의 정확성
    "completeness": "완성도",  # 내용의 완성도
    "readability": "가독성"    # 읽기 쉬움
}

print("✅ 에이전트 타입 및 상수 정의 완료")

print(f"""
[에이전트 타입]
  - RESEARCH: 정보 수집 및 분석
  - WRITER: 콘텐츠 작성 및 편집
  - CRITIC: 평가 및 피드백
  - SUPERVISOR: 전체 작업 조율

[작업 상태]
  PENDING → IN_PROGRESS → REFLECTING → REVISING → COMPLETED
  
[성찰 설정]
  - 최대 반복: {MAX_REFLECTION_ITERATIONS}회
  - 합격 기준: {PASS_THRESHOLD}점/10점
  - 평가 항목: {len(EVALUATION_CRITERIA)}개
""")


Cell 5: 에이전트 타입 및 상수 정의
✅ 에이전트 타입 및 상수 정의 완료

[에이전트 타입]
  - RESEARCH: 정보 수집 및 분석
  - WRITER: 콘텐츠 작성 및 편집
  - CRITIC: 평가 및 피드백
  - SUPERVISOR: 전체 작업 조율

[작업 상태]
  PENDING → IN_PROGRESS → REFLECTING → REVISING → COMPLETED

[성찰 설정]
  - 최대 반복: 3회
  - 합격 기준: 7점/10점
  - 평가 항목: 3개



In [5]:
print("\n" + "="*80)
print("Cell 6: Research Agent용 웹 검색 도구")
print("="*80)

# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# 웹 검색 도구 (6주차 재사용)
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

if tavily_api_key:
    # Tavily 웹 검색
    tavily_search = TavilySearchResults(
        api_key=tavily_api_key,
        max_results=5  # Research Agent는 더 많은 결과 필요
    )
    
    web_search_tool = Tool(
        name="web_search",
        description="""인터넷에서 최신 정보를 검색합니다.

사용 시기:
- 최신 뉴스, 트렌드
- 기술 동향, 연구 논문
- 사실 확인

입력: 검색 쿼리
출력: 검색 결과 리스트 (최대 5개)

예시:
- "2025 AI trends"
- "LangGraph tutorial"
""",
        func=tavily_search.invoke
    )
    
    print("✅ 웹 검색 도구 생성 완료 (Tavily)")
    
else:
    # Fallback: 더미 도구
    web_search_tool = Tool(
        name="web_search",
        description="웹 검색 (더미 버전)",
        func=lambda q: [{"content": f"'{q}'에 대한 검색 결과 (더미)"}]
    )
    print("⚠️ Tavily API 키 없음 - 더미 웹 검색 도구 사용")

print("""
[Research Agent 도구]
  - web_search: 웹 검색 (최대 5개 결과)
  - 용도: 최신 정보 수집, 트렌드 분석
  
[6주차 대비 차이]
  - max_results: 3 → 5 (더 많은 정보)
  - Research Agent 전용
""")


Cell 6: Research Agent용 웹 검색 도구
✅ 웹 검색 도구 생성 완료 (Tavily)

[Research Agent 도구]
  - web_search: 웹 검색 (최대 5개 결과)
  - 용도: 최신 정보 수집, 트렌드 분석

[6주차 대비 차이]
  - max_results: 3 → 5 (더 많은 정보)
  - Research Agent 전용



In [6]:
print("\n" + "="*80)
print("Section 0 완료 요약")
print("="*80)

print("""
✅ Section 0: 환경 설정 완료

[구현 완료 항목]
1. ✅ 라이브러리 Import
   - 6주차 재사용 + 7주차 신규 (dataclass, Enum)

2. ✅ API 키 설정
   - OpenAI (필수)
   - Tavily (Research Agent용)

3. ✅ LLM 설정
   - temperature: 0 → 0.7 (창의성)

4. ✅ 에이전트 타입 정의
   - AgentType: RESEARCH, WRITER, CRITIC, SUPERVISOR
   - TaskStatus: 작업 상태 관리

5. ✅ 성찰 상수
   - MAX_REFLECTION_ITERATIONS: 3
   - PASS_THRESHOLD: 7점

6. ✅ Research Agent 도구
   - web_search (Tavily)

[6주차 대비 변경사항]
  ✅ temperature: 0 → 0.7
  ✅ 에이전트 타입 시스템 추가
  ✅ 성찰 메커니즘 상수 추가
  ❌ calculator, weather 제거 (7주차는 콘텐츠 작성 중심)

[다음 단계]
Section 1: 전문 에이전트 클래스 정의
  - ResearchAgent
  - WriterAgent
  - CriticAgent
""")

print("\n" + "="*80)
print("현재 진행도: Section 0 완료 (15%)")
print("="*80)


Section 0 완료 요약

✅ Section 0: 환경 설정 완료

[구현 완료 항목]
1. ✅ 라이브러리 Import
   - 6주차 재사용 + 7주차 신규 (dataclass, Enum)

2. ✅ API 키 설정
   - OpenAI (필수)
   - Tavily (Research Agent용)

3. ✅ LLM 설정
   - temperature: 0 → 0.7 (창의성)

4. ✅ 에이전트 타입 정의
   - AgentType: RESEARCH, WRITER, CRITIC, SUPERVISOR
   - TaskStatus: 작업 상태 관리

5. ✅ 성찰 상수
   - MAX_REFLECTION_ITERATIONS: 3
   - PASS_THRESHOLD: 7점

6. ✅ Research Agent 도구
   - web_search (Tavily)

[6주차 대비 변경사항]
  ✅ temperature: 0 → 0.7
  ✅ 에이전트 타입 시스템 추가
  ✅ 성찰 메커니즘 상수 추가
  ❌ calculator, weather 제거 (7주차는 콘텐츠 작성 중심)

[다음 단계]
Section 1: 전문 에이전트 클래스 정의
  - ResearchAgent
  - WriterAgent
  - CriticAgent


현재 진행도: Section 0 완료 (15%)


---
# Section 1: 전문 에이전트 클래스 정의
---

## 목표
- 3개 전문 에이전트 클래스 구현
- 각 에이전트의 역할과 프롬프트 정의
- 기본 실행 함수 구현

## 3개 에이전트 역할

### 1️⃣ ResearchAgent (연구원)
**역할**: 정보 수집 및 분석 전문가

기능:
- 웹에서 최신 정보 검색
- 신뢰할 수 있는 출처 확인
- 핵심 내용 요약

사용 도구:
- web_search (Tavily)

출력 예시:
```
[주제: AI 트렌드]
- 2025년 LLM 발전: GPT-5, Claude 4 출시 예정
- 멀티모달 AI 확산: 텍스트+이미지+비디오 통합
- 출처: TechCrunch, ArXiv
```

### 2️⃣ WriterAgent (작가)
**역할**: 콘텐츠 작성 및 편집 전문가

기능:
- 자료 기반 글 작성
- 구조화 (서론-본론-결론)
- 톤과 스타일 일관성

스타일 옵션:
- formal: 공식적, 전문적
- casual: 친근한, 쉬운
- academic: 학술적, 상세한

출력 예시:
```
# 2025 AI 트렌드

## 서론
인공지능 분야는 2025년에도 급속한 발전을 이어가고 있습니다...

## 본론
1. LLM의 진화: GPT-5와 Claude 4는...
2. 멀티모달 AI: 텍스트, 이미지, 비디오를...
```

### 3️⃣ CriticAgent (비평가)
**역할**: 평가 및 피드백 전문가

기능:
- 논리적 오류 찾기
- 내용 정확성 검증
- 구체적 개선 방안 제시

평가 기준 (각 1-10점):
- accuracy: 정확성
- completeness: 완성도
- readability: 가독성

출력 예시:
```
[평가 점수]
- 정확성: 8/10
- 완성도: 6/10
- 가독성: 7/10

[피드백]
1. 완성도 부족: 예시 추가 필요
2. 통계 자료 보강 권장
3. 결론 부분 강화 필요
```

In [8]:
print("="*80)
print("Cell 9: MultiAgentState 정의")
print("="*80)

from typing import TypedDict, Annotated, List, Dict, Any, Optional
from langgraph.graph.message import add_messages
from langchain_core.messages import BaseMessage

# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# MultiAgentState 정의
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

class MultiAgentState(TypedDict):
    """다중 에이전트 시스템 State
    
    Attributes:
        messages: 대화 메시지 리스트
        task: 사용자의 원본 요청
        current_agent: 현재 작업 중인 에이전트
        agent_results: 에이전트별 결과물
        task_queue: 대기 중인 작업 목록
        reflection_count: 에이전트별 성찰 횟수
        quality_scores: 에이전트별 품질 점수
        final_output: 최종 출력물
    """
    # 기본 필드
    messages: Annotated[List[BaseMessage], add_messages]
    task: str
    
    # 에이전트 관리
    current_agent: str  # "research" | "writer" | "critic" | "supervisor"
    
    # 작업 결과
    agent_results: Dict[str, Any]  # {"research": {...}, "writer": {...}}
    task_queue: List[Dict[str, Any]]  # [{"agent": "research", "task": "..."}]
    
    # 성찰 메커니즘
    reflection_count: Dict[str, int]  # {"research": 2, "writer": 1}
    quality_scores: Dict[str, Dict[str, float]]  # {"research": {"accuracy": 8.0}}
    
    # 최종 결과
    final_output: Optional[str]

print("✅ MultiAgentState 정의 완료")

print("""
[MultiAgentState 필드 설명]

1. messages (6주차 재사용)
   - 전체 대화 히스토리

2. task
   - 사용자의 원본 요청
   - 예: "AI 트렌드 보고서 작성"

3. current_agent
   - 현재 작업 중인 에이전트
   - "research" | "writer" | "critic" | "supervisor"

4. agent_results (핵심!)
   - 각 에이전트의 작업 결과 저장
   - 예: {"research": "AI 트렌드 자료...", "writer": "초안..."}

5. task_queue
   - 대기 중인 작업 목록
   - 순차/병렬 협업 제어

6. reflection_count
   - 에이전트별 성찰 반복 횟수
   - 최대 3회 제한

7. quality_scores
   - 에이전트별 자가 평가 점수
   - {"research": {"accuracy": 8, "completeness": 7}}

8. final_output
   - 모든 작업 완료 후 최종 결과물
""")

Cell 9: MultiAgentState 정의
✅ MultiAgentState 정의 완료

[MultiAgentState 필드 설명]

1. messages (6주차 재사용)
   - 전체 대화 히스토리

2. task
   - 사용자의 원본 요청
   - 예: "AI 트렌드 보고서 작성"

3. current_agent
   - 현재 작업 중인 에이전트
   - "research" | "writer" | "critic" | "supervisor"

4. agent_results (핵심!)
   - 각 에이전트의 작업 결과 저장
   - 예: {"research": "AI 트렌드 자료...", "writer": "초안..."}

5. task_queue
   - 대기 중인 작업 목록
   - 순차/병렬 협업 제어

6. reflection_count
   - 에이전트별 성찰 반복 횟수
   - 최대 3회 제한

7. quality_scores
   - 에이전트별 자가 평가 점수
   - {"research": {"accuracy": 8, "completeness": 7}}

8. final_output
   - 모든 작업 완료 후 최종 결과물



In [9]:
print("\n" + "="*80)
print("Cell 10: ResearchAgent 클래스 정의")
print("="*80)

# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# ResearchAgent 프롬프트
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

research_prompt_template = """당신은 정보 수집 및 분석 전문가입니다.

역할:
- 웹에서 최신 정보 검색
- 신뢰할 수 있는 출처 확인
- 핵심 내용 요약

주제: {topic}

웹 검색을 통해 다음을 수행하세요:
1. 최신 정보 및 트렌드 파악
2. 신뢰할 수 있는 출처 3개 이상 확인
3. 핵심 내용을 간결하게 요약

출력 형식:
[주제: ...]
- 핵심 내용 1
- 핵심 내용 2
- 핵심 내용 3
- 출처: ...

연구 결과:"""

research_prompt = PromptTemplate(
    template=research_prompt_template,
    input_variables=["topic"]
)

# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# ResearchAgent 클래스
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

@dataclass
class ResearchAgent:
    """연구원 에이전트
    
    정보 수집 및 분석 전문가
    """
    name: str = "Research Agent"
    agent_type: AgentType = AgentType.RESEARCH
    
    def execute(self, topic: str) -> str:
        """연구 수행
        
        Args:
            topic: 연구 주제
            
        Returns:
            연구 결과 요약
        """
        print(f"\n[{self.name}] 연구 시작: '{topic}'")
        
        # 1. 웹 검색
        print(f"  [1/2] 웹 검색 중...")
        search_results = web_search_tool.invoke(topic)
        
        # 검색 결과 포맷팅
        search_text = "\n".join([
            f"- {result.get('content', result)[:200]}..."
            for result in search_results[:3]
        ])
        
        print(f"  ✅ {len(search_results)}개 결과 발견")
        
        # 2. LLM으로 요약
        print(f"  [2/2] 연구 결과 요약 중...")
        
        summary = (research_prompt | llm | StrOutputParser()).invoke({
            "topic": topic
        })
        
        print(f"  ✅ 연구 완료 ({len(summary)} characters)")
        
        return summary

print("✅ ResearchAgent 클래스 정의 완료")

# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# 테스트
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

print("\n[ResearchAgent 테스트]")
test_research_agent = ResearchAgent()
test_result = test_research_agent.execute("LangGraph tutorial")

print(f"\n[결과 미리보기]")
print(f"{test_result[:300]}...")

print("""
[ResearchAgent 특징]
  - 웹 검색 도구 활용
  - 최신 정보 수집
  - 핵심 내용 요약
  
[execute() 흐름]
  1. web_search_tool 호출
  2. 검색 결과 포맷팅
  3. LLM으로 요약 생성
""")


Cell 10: ResearchAgent 클래스 정의
✅ ResearchAgent 클래스 정의 완료

[ResearchAgent 테스트]

[Research Agent] 연구 시작: 'LangGraph tutorial'
  [1/2] 웹 검색 중...
  ✅ 5개 결과 발견
  [2/2] 연구 결과 요약 중...
  ✅ 연구 완료 (628 characters)

[결과 미리보기]
죄송하지만, 실시간 웹 검색 기능이 비활성화되어 있어 최신 정보를 직접 검색할 수는 없습니다. 그러나 LangGraph와 관련된 정보를 일반적으로 제공할 수 있습니다. LangGraph는 주로 자연어 처리(NLP)와 관련된 그래프 기반의 언어 모델링을 다루는 툴이나 라이브러리로 알려져 있습니다. 이에 대한 정보를 제공하겠습니다.

[주제: LangGraph tutorial]
- LangGraph는 자연어 처리(NLP) 작업을 지원하기 위해 설계된 그래프 기반의 모델입니다. 이 모델은 문서의 단어 간 관계를 그래프로 표현하여 복잡한 ...

[ResearchAgent 특징]
  - 웹 검색 도구 활용
  - 최신 정보 수집
  - 핵심 내용 요약

[execute() 흐름]
  1. web_search_tool 호출
  2. 검색 결과 포맷팅
  3. LLM으로 요약 생성



In [10]:
print("\n" + "="*80)
print("Cell 11: WriterAgent 클래스 정의")
print("="*80)

# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# WriterAgent 프롬프트
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

writer_prompt_template = """당신은 콘텐츠 작성 및 편집 전문가입니다.

역할:
- 제공된 자료를 바탕으로 글 작성
- 명확한 구조 (서론-본론-결론)
- 일관된 톤과 스타일 유지

스타일: {style}
- formal: 공식적, 전문적 어조
- casual: 친근하고 쉬운 표현
- academic: 학술적, 상세한 설명

주제: {topic}

참고 자료:
{reference}

위 자료를 바탕으로 {style} 스타일의 글을 작성하세요.

구조:
1. 서론: 주제 소개
2. 본론: 핵심 내용 (2-3개 섹션)
3. 결론: 요약 및 마무리

작성된 글:"""

writer_prompt = PromptTemplate(
    template=writer_prompt_template,
    input_variables=["topic", "reference", "style"]
)

# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# WriterAgent 클래스
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

@dataclass
class WriterAgent:
    """작가 에이전트
    
    콘텐츠 작성 및 편집 전문가
    """
    name: str = "Writer Agent"
    agent_type: AgentType = AgentType.WRITER
    style: str = "casual"  # "formal" | "casual" | "academic"
    
    def execute(self, topic: str, reference: str = "") -> str:
        """콘텐츠 작성
        
        Args:
            topic: 글 주제
            reference: 참고 자료 (Research Agent 결과)
            
        Returns:
            작성된 콘텐츠
        """
        print(f"\n[{self.name}] 글 작성 시작: '{topic}'")
        print(f"  스타일: {self.style}")
        
        # 참고 자료 확인
        if not reference:
            reference = "참고 자료 없음 (일반 지식 기반)"
            print(f"  ⚠️ 참고 자료 없음 - 일반 지식으로 작성")
        else:
            print(f"  ✅ 참고 자료: {len(reference)} characters")
        
        # LLM으로 글 작성
        print(f"  [1/1] LLM으로 콘텐츠 생성 중...")
        
        content = (writer_prompt | llm | StrOutputParser()).invoke({
            "topic": topic,
            "reference": reference,
            "style": self.style
        })
        
        print(f"  ✅ 작성 완료 ({len(content)} characters)")
        
        return content
    
    def set_style(self, style: str):
        """글쓰기 스타일 변경
        
        Args:
            style: "formal" | "casual" | "academic"
        """
        if style in ["formal", "casual", "academic"]:
            self.style = style
            print(f"  ✅ 스타일 변경: {style}")
        else:
            print(f"  ⚠️ 유효하지 않은 스타일: {style}")

print("✅ WriterAgent 클래스 정의 완료")

# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# 테스트
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

print("\n[WriterAgent 테스트]")
test_writer_agent = WriterAgent(style="casual")
test_content = test_writer_agent.execute(
    topic="AI의 미래",
    reference="AI는 2025년에도 빠르게 발전하고 있습니다."
)

print(f"\n[결과 미리보기]")
print(f"{test_content[:300]}...")

print("""
[WriterAgent 특징]
  - 참고 자료 기반 작성
  - 3가지 스타일 지원
  - 구조화된 콘텐츠
  
[execute() 흐름]
  1. 참고 자료 확인
  2. LLM으로 글 작성
  3. 스타일 반영
""")


Cell 11: WriterAgent 클래스 정의
✅ WriterAgent 클래스 정의 완료

[WriterAgent 테스트]

[Writer Agent] 글 작성 시작: 'AI의 미래'
  스타일: casual
  ✅ 참고 자료: 26 characters
  [1/1] LLM으로 콘텐츠 생성 중...
  ✅ 작성 완료 (1111 characters)

[결과 미리보기]
### AI의 미래: 2025년을 향해

#### 서론
안녕하세요! 오늘은 인공지능, 즉 AI의 미래에 대해 이야기해볼까 해요. 특히 2025년까지의 발전 방향에 대해 함께 생각해보면 좋을 것 같아요. AI는 이미 우리 생활 깊숙이 들어와 있고, 앞으로도 계속해서 빠르게 발전할 예정이에요. 그럼 AI가 어떤 모습으로 우리와 함께 할지 살펴볼까요?

#### 본론

**1. 일상생활의 변화**
AI는 우리의 일상에 큰 변화를 가져올 거예요. 예를 들어, 스마트 홈 기기들은 이미 많은 가정에서 사용되고 있죠. 2025년에는 이러한 기기...

[WriterAgent 특징]
  - 참고 자료 기반 작성
  - 3가지 스타일 지원
  - 구조화된 콘텐츠

[execute() 흐름]
  1. 참고 자료 확인
  2. LLM으로 글 작성
  3. 스타일 반영



In [11]:
print("\n" + "="*80)
print("Cell 12: CriticAgent 클래스 정의")
print("="*80)

# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# CriticAgent 프롬프트
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

critic_prompt_template = """당신은 콘텐츠 평가 및 피드백 전문가입니다.

역할:
- 논리적 오류 찾기
- 내용 정확성 검증
- 구체적 개선 방안 제시

평가 기준 (각 1-10점):
- accuracy: 정확성 (정보의 정확성, 사실 확인)
- completeness: 완성도 (내용의 완성도, 충분한 설명)
- readability: 가독성 (읽기 쉬움, 명확한 표현)

주제: {topic}

평가 대상 콘텐츠:
{content}

위 콘텐츠를 평가하고 다음 형식으로 출력하세요:

[평가 점수]
- 정확성: X/10
- 완성도: X/10
- 가독성: X/10

[총점]
X/10

[피드백]
1. (구체적인 개선점)
2. (구체적인 개선점)
3. (구체적인 개선점)

평가 결과:"""

critic_prompt = PromptTemplate(
    template=critic_prompt_template,
    input_variables=["topic", "content"]
)

# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# CriticAgent 클래스
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

@dataclass
class CriticAgent:
    """비평가 에이전트
    
    평가 및 피드백 전문가
    """
    name: str = "Critic Agent"
    agent_type: AgentType = AgentType.CRITIC
    
    def execute(self, topic: str, content: str) -> Dict[str, Any]:
        """콘텐츠 평가
        
        Args:
            topic: 주제
            content: 평가할 콘텐츠
            
        Returns:
            평가 결과 딕셔너리
            {
                "scores": {"accuracy": 8, "completeness": 7, "readability": 9},
                "average": 8.0,
                "feedback": "1. ...\n2. ...",
                "raw_output": "..."
            }
        """
        print(f"\n[{self.name}] 평가 시작: '{topic}'")
        print(f"  콘텐츠 길이: {len(content)} characters")
        
        # LLM으로 평가
        print(f"  [1/2] LLM으로 평가 중...")
        
        evaluation = (critic_prompt | llm | StrOutputParser()).invoke({
            "topic": topic,
            "content": content
        })
        
        print(f"  ✅ 평가 완료")
        
        # 점수 파싱
        print(f"  [2/2] 점수 파싱 중...")
        scores = self._parse_scores(evaluation)
        
        print(f"  ✅ 평균 점수: {scores['average']:.1f}/10")
        
        return {
            "scores": scores["details"],
            "average": scores["average"],
            "feedback": evaluation,
            "raw_output": evaluation
        }
    
    def _parse_scores(self, evaluation: str) -> Dict[str, Any]:
        """평가 텍스트에서 점수 추출
        
        Args:
            evaluation: 평가 텍스트
            
        Returns:
            {"details": {"accuracy": 8, ...}, "average": 8.0}
        """
        import re
        
        scores = {}
        
        # 정규식으로 점수 추출
        patterns = {
            "accuracy": r"정확성[:\s]*(\d+)/10",
            "completeness": r"완성도[:\s]*(\d+)/10",
            "readability": r"가독성[:\s]*(\d+)/10"
        }
        
        for key, pattern in patterns.items():
            match = re.search(pattern, evaluation)
            if match:
                scores[key] = int(match.group(1))
            else:
                scores[key] = 7  # 기본값
        
        average = sum(scores.values()) / len(scores)
        
        return {
            "details": scores,
            "average": average
        }

print("✅ CriticAgent 클래스 정의 완료")

# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# 테스트
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

print("\n[CriticAgent 테스트]")
test_critic_agent = CriticAgent()
test_evaluation = test_critic_agent.execute(
    topic="AI의 미래",
    content="AI는 빠르게 발전하고 있습니다. 많은 사람들이 AI를 사용합니다."
)

print(f"\n[평가 결과]")
print(f"  평균 점수: {test_evaluation['average']:.1f}/10")
print(f"  세부 점수: {test_evaluation['scores']}")
print(f"\n  피드백 미리보기:")
print(f"  {test_evaluation['feedback'][:200]}...")

print("""
[CriticAgent 특징]
  - 3가지 기준 평가
  - 구체적 피드백 제공
  - 점수 자동 파싱
  
[execute() 흐름]
  1. LLM으로 평가
  2. 점수 파싱 (정규식)
  3. 결과 구조화
""")


Cell 12: CriticAgent 클래스 정의
✅ CriticAgent 클래스 정의 완료

[CriticAgent 테스트]

[Critic Agent] 평가 시작: 'AI의 미래'
  콘텐츠 길이: 37 characters
  [1/2] LLM으로 평가 중...
  ✅ 평가 완료
  [2/2] 점수 파싱 중...
  ✅ 평균 점수: 5.0/10

[평가 결과]
  평균 점수: 5.0/10
  세부 점수: {'accuracy': 5, 'completeness': 3, 'readability': 7}

  피드백 미리보기:
  [평가 점수]
- 정확성: 5/10
- 완성도: 3/10
- 가독성: 7/10

[총점]
15/30

[피드백]
1. **정확성 향상**: AI의 발전에 대해 더욱 구체적인 사례나 통계 데이터를 제시하여 정보의 정확성을 높일 필요가 있습니다. 예를 들어, AI의 현재 기술 수준이나 특정 산업에서의 활용 사례를 언급하면 좋겠습니다.
  
2. **완성도 향상...

[CriticAgent 특징]
  - 3가지 기준 평가
  - 구체적 피드백 제공
  - 점수 자동 파싱

[execute() 흐름]
  1. LLM으로 평가
  2. 점수 파싱 (정규식)
  3. 결과 구조화

